In [1]:
### Data Preparation ###

# Uncomment to install required libraries
#!pip install dash
#!pip install plotly
#!pip install pandas

# Import required libraries
from dash import Dash, html, dcc
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Read the dataset
data = pd.read_excel("https://raw.githubusercontent.com/automat9/Business-Analytics/4345e44423e83e68e5e92815c37036670c9acad8/Semester%201/Analytics%20and%20Visualisation%20for%20Managers%20and%20Consultants/datasets/data_cleaned.xlsx",sheet_name=None)

# Assign dataframes based on sheet names
df1 = data['orders_carrier_pie']
df2 = data['carrier_late_bar']
df3 = data['top 20_late_bar']
df4 = data['orders_port_bar']
df5 = data['ontime_ahead_gauge'] 

### Data Processing ###

# Fig1: Orders handled by carrier
df1_grouped = df1.groupby('Carrier').size().reset_index(name='Order Count')

# Fig2: Chance of late delivery by carrier
carrier_totals = df2.groupby('Carrier').size().reset_index(name='Total Deliveries')
carrier_late = df2[df2['Ship Late Day count'] > 0].groupby('Carrier').size().reset_index(name='Late Deliveries')
df2_processed = pd.merge(carrier_totals, carrier_late, on='Carrier', how='left')
df2_processed['Late Deliveries'] = df2_processed['Late Deliveries'].fillna(0)
df2_processed['Late Delivery Probability'] = (df2_processed['Late Deliveries'] / df2_processed['Total Deliveries']) * 100

# Fig3: Top 20 products by late dispatch
df3_ordered = (
    df3.query('`Ship Late Day count` > 0')
    ['Product ID']
    .astype(str)
    .value_counts()
    .nlargest(20)
    .reset_index(name='Late Order Count')
    .rename(columns={'index': 'Product ID'}))
df3_ordered['colour'] = df3_ordered['Late Order Count'].apply(
    lambda x: 'lightgreen' if x < 10 else 'khaki' if 10 <= x <= 13 else 'lightcoral')

# Fig4: Orders handled by port
df4_grouped = df4.groupby('Origin Port').size().reset_index(name='Number of Orders')

# Fig5: Ship on-time/ahead of schedule
total_deliveries = len(df5)
df5_ok_deliveries = len(df5[df5['Ship Late Day count'] <= 0])
df5_ok_percentage = (df5_ok_deliveries / total_deliveries) * 100



### Initialise dash ###

app = Dash(__name__)

### Figure Setup ###

fig1 = px.pie(
    df1_grouped,
    values = 'Order Count',
    names = 'Carrier',
    title = 'Orders Handled by Carrier')

fig2 = px.bar(
    df2_processed,
    x='Carrier',
    y='Late Delivery Probability',
    title="Chance of Late Delivery by Carrier",
    labels={'Late Delivery Probability': 'Probability (%)'},
    text='Late Delivery Probability',
    color='Carrier')
fig2.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig2.update_layout(yaxis=dict(range=[0, 3.5]), coloraxis=dict(
    colorbar=dict(title='Carrier'),
    colorscale=[[0, 'blue'], [0.5, 'red'], [1, 'green']]))

fig3 = px.bar(
    df3_ordered,
    x='Product ID',
    y='Late Order Count',
    labels={'Late Order Count': 'Late Dispatch'},
    title='Products by Most Late Dispatch',
    text='Late Order Count')
fig3.update_traces(marker=dict(color=df3_ordered['colour']),texttemplate='%{text}', textposition='outside')
fig3.update_layout(yaxis=dict(range=[0, 20]))

fig4 = px.bar(
    df4_grouped,
    x = 'Origin Port',
    y = 'Number of Orders',
    title = 'Number of Orders per Origin Port',
    labels = {'Number of Orders': 'Orders'},
    text='Number of Orders')

fig5 = go.Figure(go.Indicator(
    mode='gauge+number',
    value=df5_ok_percentage,
    title={'text':'Percentage of On-Time or Ahead Deliveries'},
    gauge={
        'axis': {'range': [0, 100],'tickvals': [0, 20, 40, 60, 80, 90, 95, 100], 'tickangle':0},
        'bar': {'color': "#636EFA"},
        'steps': [
            {'range': [0, 90], 'color': "lightcoral"},
            {'range': [90,95], 'color': 'khaki'},
            {'range': [95, 100], 'color': "lightgreen"}]}))

### Layout ###

app.layout = html.Div([
    html.Div([
        html.Img(
            src='https://github.com/automat9/Business-Analytics/blob/master/Semester%201/Analytics%20and%20Visualisation%20for%20Managers%20and%20Consultants/visuals/Logistics%20Company%20Logo.png?raw=true',
            style={'height': '130px', 'width': '130px'}),
        
        html.H1('Logistics Insights Dashboard', style={'font-size': '60px', 'margin-left': '20px'})],
        style={'display': 'flex', 'align-items': 'center', 'border-bottom': '2px solid #ddd'}),

    html.Div([
        html.H3('Carrier Performance',style={'font-size': '24px', 'text-align': 'center', 'margin-bottom': '10px'}),
        html.Div([
            dcc.Graph(figure=fig1, style={'width': '50%', 'display': 'inline-block', 'padding': '10px'}),
            dcc.Graph(figure=fig2, style={'width': '50%', 'display': 'inline-block', 'padding': '10px'})], 
            style={'display': 'flex'})], 
        style={'border-bottom': '2px solid #ddd'}),

    html.Div([
        html.H3('Operational Insights', style={'font-size': '24px', 'text-align': 'center', 'margin-bottom': '10px'}),
        html.Div([
            dcc.Graph(figure=fig3, style={'width': '50%', 'display': 'inline-block', 'padding': '10px'}),
            dcc.Graph(figure=fig4, style={'width': '50%', 'display': 'inline-block', 'padding': '10px'})],
            style={'display': 'flex'})], 
        style={'border-bottom': '2px solid #ddd'}),

    html.Div([html.H3('Timely Deliveries', style={'font-size': '24px', 'text-align': 'center', 'margin-bottom': '10px'}),
        dcc.Graph(figure=fig5, style={'width': '100%'})])])

### Launch the Dashboard ###

if __name__ == '__main__':
    app.run(debug=True, port=1245)